In [62]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [63]:
test_data = pd.read_csv('test.csv')
train_data = pd.read_csv('train.csv')

In [64]:

train_data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [65]:
colums_to_keep = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked', 'Survived']
x = train_data[colums_to_keep]


In [66]:
x.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
Survived      0
dtype: int64

In [67]:
x['Age'].fillna(train_data['Age'].mean(), inplace=True)

C:\Users\LEOPARD\AppData\Local\Temp\ipykernel_7860\1242010179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Age'].fillna(train_data['Age'].mean(), inplace=True)


In [68]:
x = pd.get_dummies(x, columns=['Sex', 'Embarked'], drop_first=True)

In [69]:
y = x['Survived']
x = x.drop('Survived', axis=1)
x

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,22.000000,1,0,7.2500,True,False,True
1,1,38.000000,1,0,71.2833,False,False,False
2,3,26.000000,0,0,7.9250,False,False,True
3,1,35.000000,1,0,53.1000,False,False,True
4,3,35.000000,0,0,8.0500,True,False,True
...,...,...,...,...,...,...,...,...
886,2,27.000000,0,0,13.0000,True,False,True
887,1,19.000000,0,0,30.0000,False,False,True
888,3,29.699118,1,2,23.4500,False,False,True
889,1,26.000000,0,0,30.0000,True,False,False


In [70]:
colums_to_keep = ['Pclass','Sex','Age','SibSp','Parch','Fare', 'Embarked']
d_tarin2 = test_data[colums_to_keep]
d_tarin2['Age'].fillna(train_data['Age'].mean(), inplace=True)
d_tarin2 = pd.get_dummies(d_tarin2, columns=['Sex', 'Embarked'], drop_first=True)
d_tarin2.head()

C:\Users\LEOPARD\AppData\Local\Temp\ipykernel_7860\2133444139.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d_tarin2['Age'].fillna(train_data['Age'].mean(), inplace=True)


,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S
0,3,34.5,0,0,7.8292,True,True,False
1,3,47.0,1,0,7.0000,False,False,True
2,2,62.0,0,0,9.6875,True,True,False
3,3,27.0,0,0,8.6625,True,False,True
4,3,22.0,1,1,12.2875,False,False,True


In [71]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


In [72]:
dtrain = xgb.DMatrix(x_train, label=y_train)
dtest = xgb.DMatrix(x_test, label=y_test)

In [73]:
params = {
    'booster': 'gbtree',
    'objective': 'multi:softmax',
    'num_class': 2,
    'eta': 0.1,
    'max_depth': 5,
    'min_child_weight': 3,
    'gamma': 0.1,
    'verbosity': 1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'num_parallel_tree': 80
}

In [74]:
num_round = 1
bst = xgb.train(params, dtrain, num_round)

In [75]:
preds = bst.predict(dtest)

In [76]:
accuracy = accuracy_score(y_test, preds)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.81


In [77]:
dnew = xgb.DMatrix(d_tarin2)

In [78]:
new_preds = bst.predict(dnew)

In [82]:
passenger_ids = test_data['PassengerId']
results = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': new_preds.astype(int)  # Приведение прогнозов к целым числам
})
results

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
